In [9]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
'''
  ████
██    ██   Datature
  ██  ██   Powering Breakthrough AI
    ██

@File    :   dicom_nifti_upload.ipynb
@Author  :   Koh Quan Wei Ivan
@Version :   1.0
@Contact :   hello@datature.io
@License :   Apache License 2.0
@Desc    :   Demo for uploading DICOM and NIFTI files using Datature Python SDK.
'''


'\n  ████\n██    ██   Datature\n  ██  ██   Powering Breakthrough AI\n    ██\n\n@File    :   dicom_nifti_demo.ipynb\n@Author  :   Koh Quan Wei Ivan\n@Version :   1.0\n@Contact :   hello@datature.io\n@License :   Apache License 2.0\n@Desc    :   <Description>\n'

# Install required libraries

In [1]:
! pip install -U datature

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Import Libraries

In [4]:
import datature
import time

# Importing DICOM or NIFTI files

Nexus currently has support for the following single file uploads without any need for modification:

1. Single NIFTI files containing slices of a single volume
2. Single DICOM files containing slices 

The following cases will require some modification to the code:

3. Multiple DICOM files in a directory representing a single volume

In [3]:
# change to your own data 
nifti_file_path = "data/BRAINIX_NIFTI_Output_3D_File.nii"
dicom_file_path = "data/0009.DCM"
dicom_folder_path = "data/T2"

# Upload DICOM/NIFTI files to Datature Nexus

The steps are as follows:
1. Create a new project on Nexus and retrieve the [secret key](https://developers.datature.io/docs/hub-and-api) in the `Integrations` page.
2. Use Datature SDK to upload the DICOM/NIFTI files to Datature Nexus

For NIFTI files, each file is uploaded as a separate 3D volume.
- If the axis of orientation is provided, the SDK will upload a series of 2D slices corresponding to the specified orientation. You will only see one asset containing multiple slices on Nexus.
- If the axis of orientation is not provided, the SDK will upload a series of 2D slices for each orientation (x, y, z). This means that you will see three separate assets on Nexus.

For DICOM files, only single file uploads are supported. 
- Each single DICOM file can either represent a single image or a series of 2D slices of a 3D volume.
- This means that if you have a 3D volume from multiple stacked DICOM files, you can either:
    - Convert it to a NIFTI file for upload, or 
    - Upload DICOM files consisting of 2D series for each orientation separately.

## Example: Initialize an upload session to upload a NIFTI file (no orientation specified)

Three NIFTI files are uploaded to Datature Nexus, each corresponding to the x, y, and z orientations respectively.

In [5]:
datature.secret_key = "<your-secret-key>"
single_nifti_file_path = nifti_file_path

upload_session = datature.Asset.upload_session()

# upload the single NIFTI file containing a 3D volume
upload_session.add(
    single_nifti_file_path
)  # this will upload axial, coronal, sagittal planes separately

# start the upload session in the background
# and retrieve the operation link for status checks
op_link = upload_session.start(["main"], background=True)["op_link"]

In [6]:
# check the status of the upload
num_files = 3
while datature.Operation.retrieve(
        op_link)["status"]["progress"]["with_status"]["finished"] != num_files:
    time.sleep(1)
print(f"Uploaded '{nifti_file_path}' to Nexus!")

Uploaded 'data/BRAINIX_NIFTI_Output_3D_File.nii' to Nexus!


## Example: Initialize upload session to upload a NIFTI file (with specified orientation)

Here, we specify that we want to upload the NIFTI file with the orientation of the slices being along the x-axis. You can also specify the orientation to be along the y-axis or z-axis by changing the value of the `orientation` parameter.

In [7]:
single_nifti_file_path = nifti_file_path

upload_session = datature.Asset.upload_session()

# upload the single NIFTI file containing a 3D volume
upload_session.add(single_nifti_file_path, orientation="x")  # x orientation

# start the upload session in the background
# and retrieve the operation link for status checks
op_link = upload_session.start(['main'], background=True)["op_link"]

In [8]:
# check the status of the upload
num_files = 1
while datature.Operation.retrieve(
        op_link)["status"]["progress"]["with_status"]["finished"] != num_files:
    time.sleep(1)
print(f"Uploaded '{single_nifti_file_path}' to Nexus!")

Uploaded 'data/BRAINIX_NIFTI_Output_3D_File.nii' to Nexus!


## Example: Initialize an upload session to upload a DICOM file

In [9]:
single_dicom_file_path = dicom_file_path

upload_session = datature.Asset.upload_session()

# upload the single DICOM file containing a series of slices
upload_session.add(dicom_file_path)

# start the upload session in the background
# and retrieve the operation link for status checks
op_link = upload_session.start(['main'], background=True)["op_link"]

In [10]:
# check the status of the upload
num_files = 1
while datature.Operation.retrieve(
        op_link)["status"]["progress"]["with_status"]["finished"] != num_files:
    time.sleep(1)
print(f"Uploaded '{single_dicom_file_path}' to Nexus!")

Uploaded 'data/0009.DCM' to Nexus!
